<a href="https://colab.research.google.com/github/ChinmayP3/P4-Fake-News-Prediction/blob/main/P4_Fake_News_Prediction.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

About the Dataset:

1. id: unique id for a news article
2. title: the title of a news article
3. author: author of the news article
4. text: the text of the article; could be incomplete
5. label: a label that marks whether the news article is real or fake:
           1: Fake news
           0: real News



 

Importing the dependencies

In [43]:
import numpy as np
import pandas as pd
# re stands for Regular Expressions imp for searching text in a document
import re
# nltk: natural language tool kit
# corpus body of that text
# stopwords words that dont have much importance like "are the were  we" this that
from  nltk.corpus import stopwords
# stemming removes prefix and suffix of word and returns word
from nltk.stem.porter import PorterStemmer
# TfidfVectorizer used to convert text into feature vector
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score

In [44]:
import nltk
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

Importing Dataset 

In [45]:
# printing stopwords 
print(stopwords.words('english'))

['i', 'me', 'my', 'myself', 'we', 'our', 'ours', 'ourselves', 'you', "you're", "you've", "you'll", "you'd", 'your', 'yours', 'yourself', 'yourselves', 'he', 'him', 'his', 'himself', 'she', "she's", 'her', 'hers', 'herself', 'it', "it's", 'its', 'itself', 'they', 'them', 'their', 'theirs', 'themselves', 'what', 'which', 'who', 'whom', 'this', 'that', "that'll", 'these', 'those', 'am', 'is', 'are', 'was', 'were', 'be', 'been', 'being', 'have', 'has', 'had', 'having', 'do', 'does', 'did', 'doing', 'a', 'an', 'the', 'and', 'but', 'if', 'or', 'because', 'as', 'until', 'while', 'of', 'at', 'by', 'for', 'with', 'about', 'against', 'between', 'into', 'through', 'during', 'before', 'after', 'above', 'below', 'to', 'from', 'up', 'down', 'in', 'out', 'on', 'off', 'over', 'under', 'again', 'further', 'then', 'once', 'here', 'there', 'when', 'where', 'why', 'how', 'all', 'any', 'both', 'each', 'few', 'more', 'most', 'other', 'some', 'such', 'no', 'nor', 'not', 'only', 'own', 'same', 'so', 'than', '

In [46]:
news_dataset=pd.read_csv('/content/train.csv')

In [47]:
news_dataset.head()

,id,title,author,text,label
0,0,House Dem Aide: We Didn’t Even See Comey’s Let...,Darrell Lucus,House Dem Aide: We Didn’t Even See Comey’s Let...,1
1,1,"FLYNN: Hillary Clinton, Big Woman on Campus - ...",Daniel J. Flynn,Ever get the feeling your life circles the rou...,0
2,2,Why the Truth Might Get You Fired,Consortiumnews.com,"Why the Truth Might Get You Fired October 29, ...",1
3,3,15 Civilians Killed In Single US Airstrike Hav...,Jessica Purkiss,Videos 15 Civilians Killed In Single US Airstr...,1
4,4,Iranian woman jailed for fictional unpublished...,Howard Portnoy,Print \nAn Iranian woman has been sentenced to...,1


In [48]:
news_dataset.describe()

,id,label
count,20800.000000,20800.000000
mean,10399.500000,0.500625
std,6004.587135,0.500012
min,0.000000,0.000000
25%,5199.750000,0.000000
50%,10399.500000,1.000000
75%,15599.250000,1.000000
max,20799.000000,1.000000


In [49]:
news_dataset.shape

(20800, 5)

In [50]:
news_dataset.isnull().sum()

id           0
title      558
author    1957
text        39
label        0
dtype: int64

In [51]:
# here we can see that there are many columns with  NULL i.e missing 
# Now bcz no of missing is less , we will drop them , 
# else if no is higher we will do Imputation
# Its is a technique used for replacing the missing data with some substitute value (like mean)to retain most of the data/information of the dataset.

In [52]:
# Replacing missing values with NA
# we use fillna to fill '' nothing in place of missing values
news_dataset=news_dataset.fillna('')

In [53]:
# merging the title and author  into one column named content
news_dataset['content']=news_dataset['author']+' '+news_dataset['title'] 

In [54]:
print(news_dataset['content'])

0        Darrell Lucus House Dem Aide: We Didn’t Even S...
1        Daniel J. Flynn FLYNN: Hillary Clinton, Big Wo...
2        Consortiumnews.com Why the Truth Might Get You...
3        Jessica Purkiss 15 Civilians Killed In Single ...
4        Howard Portnoy Iranian woman jailed for fictio...
                               ...                        
20795    Jerome Hudson Rapper T.I.: Trump a ’Poster Chi...
20796    Benjamin Hoffman N.F.L. Playoffs: Schedule, Ma...
20797    Michael J. de la Merced and Rachel Abrams Macy...
20798    Alex Ansary NATO, Russia To Hold Parallel Exer...
20799              David Swanson What Keeps the F-35 Alive
Name: content, Length: 20800, dtype: object


In [55]:
X=news_dataset.drop(columns='label', axis=1)
X.head()
Y=news_dataset['label']
Y.head()

0    1
1    0
2    1
3    1
4    1
Name: label, dtype: int64

Stemming 

Stemming is a process to make all its words into root words 

eg : (acting , actress , actor) --> act

In [56]:
port_stem=PorterStemmer()

In [57]:
def stemming(content):
  # removes all form content except letters from a-z and A-Z
  # here ' ' means all removed(num, commas etc) will be replaced by blank
  stemmed_content = re.sub('[^a-zA-Z]',' ',content)
  # now converting all alphabets to lower case  
  stemmed_content = stemmed_content.lower()
  # splitting  it into list
  stemmed_content = stemmed_content.split()
  # stemming all the words except those who are in stopwords and removing them
  stemmed_content = [port_stem.stem(word) for word in stemmed_content if not word in stopwords.words('english')]
  # Joining words again ...as before we have splitted them
  stemmed_content = ' '.join(stemmed_content)
  # returning stemmed data
  return stemmed_content



In [58]:
news_dataset['content']=news_dataset['content'].apply(stemming)

In [59]:
print(news_dataset['content'])

0        darrel lucu hous dem aid even see comey letter...
1        daniel j flynn flynn hillari clinton big woman...
2                   consortiumnew com truth might get fire
3        jessica purkiss civilian kill singl us airstri...
4        howard portnoy iranian woman jail fiction unpu...
                               ...                        
20795    jerom hudson rapper trump poster child white s...
20796    benjamin hoffman n f l playoff schedul matchup...
20797    michael j de la merc rachel abram maci said re...
20798    alex ansari nato russia hold parallel exercis ...
20799                            david swanson keep f aliv
Name: content, Length: 20800, dtype: object


In [60]:
# separating data and labels
X=news_dataset['content'].values
Y=news_dataset['label'].values


In [61]:
print(X)

['darrel lucu hous dem aid even see comey letter jason chaffetz tweet'
 'daniel j flynn flynn hillari clinton big woman campu breitbart'
 'consortiumnew com truth might get fire' ...
 'michael j de la merc rachel abram maci said receiv takeov approach hudson bay new york time'
 'alex ansari nato russia hold parallel exercis balkan'
 'david swanson keep f aliv']


In [62]:
# converting text data into numerical data
# As our computer cant understand text data we should convert it into numerical data 
# here TfidfVectorizer counts repeating nos, and assign num to it acc to priority 
vectorizer = TfidfVectorizer()
vectorizer.fit(X)

X=vectorizer.transform(X)



In [63]:
print(X)

  (0, 15686)	0.28485063562728646
  (0, 13473)	0.2565896679337957
  (0, 8909)	0.3635963806326075
  (0, 8630)	0.29212514087043684
  (0, 7692)	0.24785219520671603
  (0, 7005)	0.21874169089359144
  (0, 4973)	0.233316966909351
  (0, 3792)	0.2705332480845492
  (0, 3600)	0.3598939188262559
  (0, 2959)	0.2468450128533713
  (0, 2483)	0.3676519686797209
  (0, 267)	0.27010124977708766
  (1, 16799)	0.30071745655510157
  (1, 6816)	0.1904660198296849
  (1, 5503)	0.7143299355715573
  (1, 3568)	0.26373768806048464
  (1, 2813)	0.19094574062359204
  (1, 2223)	0.3827320386859759
  (1, 1894)	0.15521974226349364
  (1, 1497)	0.2939891562094648
  (2, 15611)	0.41544962664721613
  (2, 9620)	0.49351492943649944
  (2, 5968)	0.3474613386728292
  (2, 5389)	0.3866530551182615
  (2, 3103)	0.46097489583229645
  :	:
  (20797, 13122)	0.2482526352197606
  (20797, 12344)	0.27263457663336677
  (20797, 12138)	0.24778257724396507
  (20797, 10306)	0.08038079000566466
  (20797, 9588)	0.174553480255222
  (20797, 9518)	0.295420

Splitting our data into train and test data

In [64]:
X_train, X_test, Y_train, Y_test=train_test_split(X,Y, test_size=0.2,stratify=Y,random_state=2)

In [65]:
model=LogisticRegression()

In [66]:
model.fit(X_train,Y_train)

LogisticRegression()

Evaluation 

Accuracy Score

In [67]:
X_train_prediction=model.predict(X_train)
training_data_accuracy=accuracy_score(X_train_prediction,Y_train)

In [68]:
print("Accuracy on train data is : ",training_data_accuracy)

Accuracy on train data is :  0.9865985576923076


In [69]:
X_test_prediction=model.predict(X_test)
testing_data_accuracy=accuracy_score(X_test_prediction,Y_test)

In [70]:
print("Accuracy on test data is : ",testing_data_accuracy)

Accuracy on test data is :  0.9790865384615385


Making a Predictive System

In [71]:
X_new = X_test[0]
prediction = model.predict(X_new)
print(prediction)

if(prediction[0]==0):
  print("This news is Real")
else:
  print("This news is Fake")


[1]
This news is Fake


In [72]:
print(Y_test[0])

1
